In [85]:
#coding: utf-8
import numpy as np
import itertools
from scipy import optimize
from sklearn import datasets
from sklearn import neighbors
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances
import pulp

### 線形計画問題　最大化

In [2]:
problem = pulp.LpProblem('hamburg and omelet', pulp.LpMaximize)

# create variables
x = pulp.LpVariable('x', cat = 'Integer')
y = pulp.LpVariable('y', cat = 'Integer')

# maximize
problem += 400 * x + 300 * y

# subjects
problem += 60 * x + 40 * y <= 3800
problem += 20 * x + 30 * y <= 2100
problem += 20 * x + 10 * y <= 1200

In [5]:
print(problem)

hamburg and omelet:
MAXIMIZE
400*x + 300*y + 0
SUBJECT TO
_C1: 60 x + 40 y <= 3800

_C2: 20 x + 30 y <= 2100

_C3: 20 x + 10 y <= 1200

VARIABLES
x free Integer
y free Integer



In [7]:
status = problem.solve()
print(pulp.LpStatus[status])

Optimal


In [8]:
print("Result")
print("x", x.value())
print("y", y.value())

Result
x 30.0
y 50.0


In [9]:
max_value = 400 * x.value() + 300 * y.value()
print("MAX VALUE", "400 * %d + 300 * %d = %d" % (x.value(), y.value(), max_value))

MAX VALUE 400 * 30 + 300 * 50 = 27000


### scipy

#### 簡単な例
- 共役勾配法のfmin_cg()
- BFGSアルゴリズムのfmin_bfgs()

In [13]:
def J(theta, *args):
    """最小化を目指すコスト関数を返す"""
    theta1, theta2 = theta
    return (theta1 - 5) ** 2 + (theta2 - 5) ** 2

def gradient(theta, *args):
    """コスト関数の偏微分を返す各パラメータで偏微分した関数リストを返す"""
    theta1, theta2 = theta
    # Jをtheta1で偏微分した関数
    gt1 = 2 * (theta1 - 5)
    # Jをtheta2で偏微分した関数
    gt2 = 2 * (theta2 - 5)
    return np.array((gt1, gt2))

# パラメータの初期値
initial_theta = np.array([0, 0])

In [14]:
# Conjugate Gradientによるコスト関数最小化
theta = optimize.fmin_cg(J, initial_theta, fprime=gradient)
print("conjugate gradient:", theta)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5
conjugate gradient: [ 4.99999995  4.99999995]


In [15]:
# BFGSによるコスト関数最小化
theta = optimize.fmin_bfgs(J, initial_theta, fprime=gradient)
print("BFGS:", theta)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 4
         Gradient evaluations: 4
BFGS: [ 5.  5.]


#### 違う例

In [19]:
def J(param, *args):
    """最小化を目指すコスト関数を返す"""
    u, v = param
    # パラメータ以外のデータはargsを通して渡す
    a, b, c, d, e, f = args
    return a*u**2 + b*u*v + c*v**2 + d*u + e*v + f

def gradient(param, *args):
    """コスト関数の偏微分を返す各パラメータで偏微分した関数リストを返す"""
    u, v = param
    a, b, c, d, e, f = args
    gu = 2*a*u + b*v + d
    gv = b*u + 2*c*v + e
    return np.array((gu, gv))

# パラメータの初期値
initial_param = np.array([0, 0])
args = (2, 3, 7, 8, 9, 10)

In [17]:
# Conjugate Gradientによるコスト関数最小化
param = optimize.fmin_cg(J, initial_param, fprime=gradient, args=args)
print("conjugate gradient:", param)

Optimization terminated successfully.
         Current function value: 1.617021
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5
conjugate gradient: [-1.80851064 -0.25531915]


In [18]:
# BFGSによるコスト関数最小化
param = optimize.fmin_bfgs(J, initial_param, fprime=gradient, args=args)
print("BFGS:", param)

Optimization terminated successfully.
         Current function value: 1.617021
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5
BFGS: [-1.80851064 -0.25531915]


In [25]:
def objectiveFunction(x):
    # 目的関数
    # f(x,y) = 3*x1^2 + 2*x1*x2 + x2^2
    f = 3*x[0]**2 + 2*x[0]*x[1] + x[1]**2
    return f

def gradient(x):
    # 勾配ベクトル
    g = np.array([6*x[0] + 2*x[1], 2*x[0] + 2*x[1]])
    return g

def Hessian(x):
    # ヘッセ行列
    h = np.array([[6, 2],[2, 2]])
    return h

x0 = [10.0, 10.0]
res = optimize.minimize(objectiveFunction, x0, jac=gradient, hess=Hessian, method="trust-ncg")
print(res)

     fun: 1.4791141972893971e-31
    hess: array([[6, 2],
       [2, 2]])
     jac: array([ -4.44089210e-16,   4.44089210e-16])
 message: 'Optimization terminated successfully.'
    nfev: 6
    nhev: 5
     nit: 5
    njev: 6
  status: 0
 success: True
       x: array([ -2.22044605e-16,   4.44089210e-16])


In [30]:
res = optimize.minimize(objectiveFunction, x0, jac=gradient, method="bfgs")
print(res)

      fun: 1.232595164407831e-32
 hess_inv: array([[ 0.25, -0.25],
       [-0.25,  0.75]])
      jac: array([ -2.22044605e-16,  -2.22044605e-16])
  message: 'Optimization terminated successfully.'
     nfev: 5
      nit: 3
     njev: 5
   status: 0
  success: True
        x: array([  0.00000000e+00,  -1.11022302e-16])


In [60]:
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target

In [61]:
X.shape

(150, 2)

In [63]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### 距離学習の練習

In [44]:
M = np.array([[1,0],[0,1]])
M

array([[1, 0],
       [0, 1]])

In [48]:
euclidean_distances(X, X)

array([[ 0.        ,  0.53851648,  0.5       , ...,  1.48660687,
         1.1045361 ,  0.94339811],
       [ 0.53851648,  0.        ,  0.28284271, ...,  1.6       ,
         1.36014705,  1.        ],
       [ 0.5       ,  0.28284271,  0.        , ...,  1.81107703,
         1.5132746 ,  1.21655251],
       ..., 
       [ 1.48660687,  1.6       ,  1.81107703, ...,  0.        ,
         0.5       ,  0.6       ],
       [ 1.1045361 ,  1.36014705,  1.5132746 , ...,  0.5       ,
         0.        ,  0.5       ],
       [ 0.94339811,  1.        ,  1.21655251, ...,  0.6       ,
         0.5       ,  0.        ]])

In [55]:
pairwise_distances(X, Y=None, metric='euclidean', n_jobs=1)

array([[ 0.        ,  0.53851648,  0.5       , ...,  1.48660687,
         1.1045361 ,  0.94339811],
       [ 0.53851648,  0.        ,  0.28284271, ...,  1.6       ,
         1.36014705,  1.        ],
       [ 0.5       ,  0.28284271,  0.        , ...,  1.81107703,
         1.5132746 ,  1.21655251],
       ..., 
       [ 1.48660687,  1.6       ,  1.81107703, ...,  0.        ,
         0.5       ,  0.6       ],
       [ 1.1045361 ,  1.36014705,  1.5132746 , ...,  0.5       ,
         0.        ,  0.5       ],
       [ 0.94339811,  1.        ,  1.21655251, ...,  0.6       ,
         0.5       ,  0.        ]])

In [56]:
pairwise_distances(X, Y=X, metric='euclidean')

array([[ 0.        ,  0.53851648,  0.5       , ...,  1.48660687,
         1.1045361 ,  0.94339811],
       [ 0.53851648,  0.        ,  0.28284271, ...,  1.6       ,
         1.36014705,  1.        ],
       [ 0.5       ,  0.28284271,  0.        , ...,  1.81107703,
         1.5132746 ,  1.21655251],
       ..., 
       [ 1.48660687,  1.6       ,  1.81107703, ...,  0.        ,
         0.5       ,  0.6       ],
       [ 1.1045361 ,  1.36014705,  1.5132746 , ...,  0.5       ,
         0.        ,  0.5       ],
       [ 0.94339811,  1.        ,  1.21655251, ...,  0.6       ,
         0.5       ,  0.        ]])

In [83]:
dist = neighbors.DistanceMetric.get_metric('mahalanobis', V = M)
dist.pairwise(X, X)

array([[ 0.        ,  0.53851648,  0.5       , ...,  1.48660687,
         1.1045361 ,  0.94339811],
       [ 0.53851648,  0.        ,  0.28284271, ...,  1.6       ,
         1.36014705,  1.        ],
       [ 0.5       ,  0.28284271,  0.        , ...,  1.81107703,
         1.5132746 ,  1.21655251],
       ..., 
       [ 1.48660687,  1.6       ,  1.81107703, ...,  0.        ,
         0.5       ,  0.6       ],
       [ 1.1045361 ,  1.36014705,  1.5132746 , ...,  0.5       ,
         0.        ,  0.5       ],
       [ 0.94339811,  1.        ,  1.21655251, ...,  0.6       ,
         0.5       ,  0.        ]])

In [99]:
np.sum(np.ravel(dist.pairwise(X, X)))

25745.993307778244

In [91]:
X_1 = X[y == 0]
X_2 = X[y == 1]
X_3 = X[y == 2]
X_all = list(itertools.combinations([X_1, X_2, X_3], 2))

### 距離学習の目的関数

In [112]:
def objectiveFunction(X_all, M):
    g = 0.0
    dist = neighbors.DistanceMetric.get_metric('mahalanobis', V = M)
    for pairs in X_all :
        dist_matrix = dist.pairwise(pairs[0],pairs[1])
        g += np.sum(np.ravel(dist_matrix))
    return g

In [113]:
objectiveFunction(X_all, M)

10146.028202844023

### 制約条件の関数

In [117]:
X_all = [X_1, X_2, X_3]
def constraintFunction(X_all, M):
    f = 0.0
    dist = neighbors.DistanceMetric.get_metric('mahalanobis', V = M)
    for X_same_class in X_all :
        dist_matrix = dist.pairwise(X_same_class)
        f += np.sum(dist_matrix * dist_matrix)
    return f

In [118]:
constraintFunction(X_all, M)

5599.1200000000008

### フロベニウスノルム

In [110]:
def getFrobenius(M):
    np.sum(M * M)